# Lesson 3: Chains
<a target="_blank" href="https://colab.research.google.com/github/bqtankiet/langchain-llm-course/blob/main/L3_Chains.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

This notebook documents my learning journey on **LangChain for LLM Application Development** course from Deeplearning.ai \
[Lesson 3: Chains](https://learn.deeplearning.ai/courses/langchain/lesson/glsn3/chains)

\
What I Learned
- How to create a simple chain with LangChain Expression Language (LCEL) using the | (pipe) operator.
- How to build a Sequential Chain where the output of one component becomes the input of the next.
- How to execute chains in parallel using `RunnableParallel` to create a Parallel Chain.
- How to implement a Router Chain with `RunnableBranch` to dynamically route inputs to different chains based on the input.


## Setting up the Environment

In [ ]:
!pip install -qU python-dotenv
!pip install -qU langchain-groq

In [ ]:
import os

from dotenv import load_dotenv
_ = load_dotenv() # read local .env file

In [ ]:
from langchain.chat_models import init_chat_model

llm = init_chat_model(
    model = "llama-3.3-70b-versatile",
    model_provider = "groq",
    temperature = 0.9
)

In [ ]:
import pandas as pd
df = pd.read_csv('data.csv')

In [ ]:
df

,Product,Review
0,Queen Size Sheet Set,I ordered a king size set. My only criticism w...
1,Waterproof Phone Pouch,"I loved the waterproof sac, although the openi..."
2,Luxury Air Mattress,This mattress had a small hole in the top of i...
3,Pillows Insert,This is the best throw pillow fillers on Amazo...
4,Milk Frother Handheld,I loved this product. But they only seem to l...
5,L'Or Espresso Café,Je trouve le goût médiocre. La mousse ne tient...
6,Hervidor de Agua Eléctrico,"Está lu bonita calienta muy rápido, es muy fun..."
7,Bánh kem ABC,Hương vị quá tệ! Lớp kem thì chảy nước. Chắc c...


## I. LangChain Expression Language
Build a simple LCEL chain: prompt | model | output parser

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers.string import StrOutputParser

In [ ]:
prompt = ChatPromptTemplate.from_template(
    "What is the best name to describe \
    a company that makes {product}?. \
    Respond only one company name"
)

In [ ]:
parser = StrOutputParser()

In [ ]:
chain = prompt | llm | parser # LCEL

In [ ]:
# just see what type it is
print(type(chain))

<class 'langchain_core.runnables.base.RunnableSequence'>


In [ ]:
product = "Queen Size Sheet Set"
chain.invoke({"product": product})

'RoyalSlumber'

## II. Sequential Chain
Build a two-step chain: generate a company name → write a short company description \
(step 2 depends on step 1)

In [ ]:
# prompt template 1
first_prompt = ChatPromptTemplate.from_template(
    "What is the best name to describe \
    a company that makes {product}?"
)

chain_1 = first_prompt | llm

In [ ]:
# prompt template 2
second_prompt = ChatPromptTemplate.from_template(
    "Write a 20 words description for the following \
    company:{company_name}"
)

chain_2 = second_prompt | llm

In [ ]:
overall_chain = (
    {"company_name": chain_1}
    | chain_2
    | parser
)

In [ ]:
overall_chain.invoke({"product": product})

'Luxurious bedding company offering high-quality queen size sheet sets for optimal comfort and restful sleep experiences always.'

## III. Parallel Chain

Set up the prompt templates used to:
1. Translate the review

2. Detect the language

3. Summarize the review

4. Generate a follow-up response


In [ ]:
# prompt template 1: Translate the review
prompt_1 = ChatPromptTemplate.from_template(
    "Translate the following review to English, "
    "just respond the translated review:"
    "\n\n{review}"
)
chain_translate = prompt_1 | llm | parser

In [ ]:
# prompt template 2: Detect the language
prompt_2 = ChatPromptTemplate.from_template(
    "What language is the following review, "
    "just respond the language:"
    "\n\n{review}"
)
chain_language = prompt_2 | llm | parser

In [ ]:
# prompte template 3: Summarize the review
prompt_3 = ChatPromptTemplate.from_template(
    "Summarize the following review in 1 sentence, "
    "just respond the summarized sentence:"
    "\n\n{english_review}"
)
chain_summary = prompt_3 | llm | parser

In [ ]:
# prompt template 4: Generate a follow-up response
prompt_4 = ChatPromptTemplate.from_template(
    "With the given summary of the customer review, "
    "write a follow-up response in the specified language. "
    "Just return the response sentence"
    "\n\nSummary: {summary}\n\nLanguage: {language}"
)
chain_followup = prompt_4 | llm | parser

Full pipeline: translate, detect language, summarize, and respond

In [ ]:
from langchain_core.runnables import RunnableParallel
from operator import itemgetter

In [ ]:
# approach_1: keep all results from each step
approach_1 = (
  RunnableParallel(
    english_review=chain_translate,
    language=chain_language,
  ).assign(
      summary=chain_summary
  ).assign(
      followup_message=chain_followup
  )
) # The output is a dict containing all step results

In [ ]:
# approach_2: only pass needed results to the next step
approach_2 = (
    RunnableParallel(
      english_review=chain_translate,
      language=chain_language,
    )
    | {
        "summary": chain_summary,
        "language": itemgetter("language")
    }
    | chain_followup
) # Only keep the final output

Run both approaches on a sample review

In [ ]:
review = df.Review[7]
review

'Hương vị quá tệ! Lớp kem thì chảy nước. Chắc chắn là hàng kém chất lượng hoặc bị để lâu rồi!'

In [ ]:
approach_1.invoke(review)

{'english_review': "The taste is terrible! The cream layer is watery. It's definitely low-quality goods or has been left out for too long!",
 'language': 'Vietnamese',
 'summary': 'The reviewer strongly dislikes the product, citing a terrible taste, watery cream layer, and suspecting it may be low-quality or spoiled.',
 'followup_message': 'Xin lỗi vì sản phẩm của chúng tôi không đáp ứng được nhu cầu và mong đợi của quý khách, chúng tôi sẽ xem xét lại chất lượng và quy trình sản xuất để cải thiện sản phẩm trong tương lai.'}

In [ ]:
approach_2.invoke(review)

'Xin lỗi vì sản phẩm không đáp ứng được kỳ vọng của quý khách, chúng tôi sẽ xem xét lại chất lượng và quy trình bảo quản để cải thiện sản phẩm trong tương lai.'

## IV. Router Chain

In [ ]:
physics_template = """You are a very smart physics professor. \
You are great at answering questions about physics in a concise\
and easy to understand manner. \
When you don't know the answer to a question you admit\
that you don't know.

Here is a question:
{question}"""

math_template = """You are a very good mathematician. \
You are great at answering math questions. \
You are so good because you are able to break down \
hard problems into their component parts,
answer the component parts, and then put them together\
to answer the broader question.

Here is a question:
{question}"""

history_template = """You are a very good historian. \
You have an excellent knowledge of and understanding of people,\
events and contexts from a range of historical periods. \
You have the ability to think, reflect, debate, discuss and \
evaluate the past. You have a respect for historical evidence\
and the ability to make use of it to support your explanations \
and judgements.

Here is a question:
{question}"""

computerscience_template = """ You are a successful computer scientist.\
You have a passion for creativity, collaboration,\
forward-thinking, confidence, strong problem-solving capabilities,\
understanding of theories and algorithms, and excellent communication \
skills. You are great at answering coding questions. \
You are so good because you know how to solve a problem by \
describing the solution in imperative steps \
that a machine can easily interpret and you know how to \
choose a solution that has a good balance between \
time complexity and space complexity.

Here is a question:
{question}"""

default_template = "{question}"

expert_templates = {
    "physics": physics_template,
    "math": math_template,
    "history": history_template,
    "computerscience": computerscience_template,
    "default": default_template
}

In [ ]:
from langchain_core.prompts import ChatPromptTemplate, HumanMessagePromptTemplate, SystemMessagePromptTemplate
from langchain_core.runnables import RunnableBranch, RunnableParallel
from operator import itemgetter

experts = {}
for key, template in expert_templates.items():
  prompt = ChatPromptTemplate.from_template(template)
  experts[key] = prompt | llm | parser

classifier_prompt = ChatPromptTemplate.from_messages([
    # system
    SystemMessagePromptTemplate.from_template("""
      Classify the question into one of the following topics:
      - math: problems related to algebra, calculus, statistics, equations, and general mathematics.
      - physics: topics in mechanics, electromagnetism, quantum physics, thermodynamics, etc.
      - history: historical events, time periods, historical figures, and related context.
      - computerscience: programming, algorithms, data structures, complexity, and general CS topics.
      - default: any question that does not fit the above categories.
      Only respond with one keyword: {keywords}."""),
    # human
    HumanMessagePromptTemplate.from_template("Question: {question}")
])

partial_prompt = classifier_prompt.partial(keywords=list(experts.keys()))

classifier_chain = partial_prompt | llm | parser

router_branch = RunnableBranch(
    (lambda x: "math" in x["topic"], experts["math"]),
    (lambda x: "physics" in x["topic"], experts["physics"]),
    (lambda x: "history" in x["topic"], experts["history"]),
    (lambda x: "computerscience" in x["topic"], experts["computerscience"]),
    experts["default"]
)

full_router_chain = RunnableParallel(
    {"topic": classifier_chain,
     "question": itemgetter("question")}
    ).assign(answer=router_branch)

def route_question(question: str) -> str:
    return full_router_chain.invoke({"question": question})

In [ ]:
if __name__ == "__main__":
    questions = [
        "Who are you?",
        "Solve square(x) = 4",
        "Explain langchain in short"
    ]

    for q in questions:
        print("="*100)
        response = route_question(q)
        print(f"Question: {q}")
        print(f"Topic: {response['topic']}")
        print(f"Answer: {response['answer']}")

Question: Who are you?
Topic: default
Answer: I'm an artificial intelligence model known as Llama. Llama stands for "Large Language Model Meta AI."
Question: Solve square(x) = 4
Topic: math
Answer: To solve the equation square(x) = 4, let's break it down into its component parts.

The equation square(x) = 4 can be rewritten as x^2 = 4, since "square(x)" means x squared.

Now, to solve for x, we need to find the values of x that make the equation true. 

To do this, we can take the square root of both sides of the equation:

x^2 = 4
x = ±√4
x = ±2

So, the solutions to the equation are x = 2 and x = -2.

Therefore, the final answer is x = 2 or x = -2.
Question: Explain langchain in short
Topic: computerscience
Answer: **LangChain: A Framework for Building AI-Powered Applications**

LangChain is an open-source framework that enables developers to build applications powered by large language models (LLMs). It provides a modular and flexible architecture for integrating LLMs into various a

* * *
# Summary of Your LangChain Learning Journey: Chains

In this notebook, you've embarked on a comprehensive exploration of **Chains** in LangChain, a core component for building complex LLM-powered workflows. Here's a recap of the key concepts you've mastered:

**1. LangChain Expression Language (LCEL):**

*   You started with the foundation of chain-building: the **LangChain Expression Language (LCEL)**. You learned how to use the intuitive `|` (pipe) syntax to link components (prompts, models, output parsers) together, creating simple yet powerful chains. This composition of components using the `|` operator results in a `RunnableSequence` object, which is the cornerstone of creating more complex chains.

**2. Sequential Chains:**

*   You took the next step by building **sequential chains**, where the output of one chain becomes the input for the next. This allowed you to create multi-step workflows, such as generating a company name and then writing a description for it.

**3. Parallel Chains:**

*   You explored the power of parallel execution with `RunnableParallel`. You learned how to run multiple independent chains at once and merge their results into a single object. This technique is invaluable for tasks that require multiple pieces of information to be generated simultaneously from a single input.

**4. Router Chains:**

*   Finally, you delved into advanced chaining by implementing a **Router Chain**. Using `RunnableBranch`, you created an intelligent system that can classify an input question and dynamically route it to the appropriate "expert" (a specialized chain) for an answer.

**Key Takeaways:**

Your journey through this notebook has equipped you with the skills to:

*   **Grasp** the syntax and power of the LangChain Expression Language (LCEL) for fluently creating chains.
*   **Construct** complex workflows by combining chains sequentially and in parallel.
*   **Implement** dynamic routing logic to create decision-making applications that choose the right execution path based on the input.
*   **Confidently combine** various LangChain components to build sophisticated and intricate LLM applications.

You are now well-prepared to architect and build more diverse and powerful applications with LangChain. Excellent work!

* * *
# Key Commands and Imports to Remember

### Python Libraries:
- **`import os`**: Interacts with the operating system, mainly for accessing environment variables.
- **`from dotenv import load_dotenv`**: Loads environment variables from a `.env` file to securely manage API credentials.
- **`from operator import itemgetter`**: A helper function used to fetch specific items from an input dictionary, often used within chains.

### LangChain Libraries:
- **`from langchain.chat_models import init_chat_model`**: Easily initializes a chat model instance from a provider.
- **`from langchain_core.prompts import ChatPromptTemplate`**: For creating and formatting prompt templates that can be used in a chain.
- **`from langchain_core.output_parsers.string import StrOutputParser`**: A simple parser to convert the chat model's message output into a string.
- **`from langchain_core.runnables import RunnableParallel, RunnableBranch`**: Core components for building advanced chains that execute in parallel or follow conditional logic.

### Key LangChain Concepts and Syntax:
- **`|` (pipe operator)**: The fundamental syntax of the LangChain Expression Language (LCEL). It's used to chain runnables together, passing the output of one component as the input to the next.
- **`ChatPromptTemplate.from_template()`**: A convenient method to create a prompt template from a simple string.
- **`prompt.partial()`**: A method to pre-fill some variables in a prompt template, which is useful for creating specialized prompts.
- **`RunnableParallel()`**: A runnable that executes other runnables in parallel. It takes a dictionary where values are runnables and returns a dictionary with the results.
- **`Runnable.assign()`**: A method to add or modify the output of a chain by running an additional runnable and adding its result to the output dictionary.
- **`RunnableBranch()`**: A runnable used to create conditional logic within a chain, allowing you to route the flow of execution based on the input.